In [11]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [12]:
from elasticsearch import Elasticsearch

In [13]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)

In [15]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [16]:
from tqdm.auto import tqdm

In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [18]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma2',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

"Yes, you can still join the course.  You're still eligible to submit the homeworks even if you don't register. \n"

In [23]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

prompt = "What's the formula for energy?"

response = client.chat.completions.create(
    model="gemma2",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.0
)

print(response.choices[0].message.content)

The most famous formula for energy is **E=mc²**, where:

* **E** represents energy
* **m** represents mass
* **c** represents the speed of light in a vacuum (approximately 299,792,458 meters per second)

This equation, developed by Albert Einstein, shows that **mass and energy are equivalent and interchangeable**.  

However, there are other formulas for energy depending on the specific type of energy you're interested in:

* **Kinetic energy (energy of motion):** KE = 1/2 * m * v² 
    * Where KE is kinetic energy, m is mass, and v is velocity.
* **Potential energy (stored energy):** PE = m * g * h
    * Where PE is potential energy, m is mass, g is acceleration due to gravity, and h is height.
* **Thermal energy (heat):** Q = m * c * ΔT
    * Where Q is heat energy, m is mass, c is specific heat capacity, and ΔT is the change in temperature.

Let me know if you'd like to explore a specific type of energy in more detail!


In [25]:
print(f"Completion tokens: {response.usage.completion_tokens}")

Completion tokens: 252


In [26]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

prompt = "10 * 10"

response = client.chat.completions.create(
    model="gemma2",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.0
)

print(response.choices[0].message.content)

10 * 10 = 100 

